# This notebook is the hand implementation of the regression methods such as Lasso, Elastic Net and Polynomial Ridge Regression

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Help functions for regression models

In [2]:
def add_poly_features(X, degree):
    """ This function adds additional polynomial features to data """
    
    n_samples_in, n_features_in = np.shape(X)
    
    combs_of_deg = [combinations_with_replacement(range(n_features_in), i) for i in range(0, degree + 1)]
    deg_combs = [deg for deglist in combs_of_deg for deg in deglist]
    
    n_features_out = len(deg_combs)
    X_out = np.empty((n_samples_in, n_features_out))
    
    for i, deg_pair in enumerate(deg_combs):  
        X_out[:, i] = np.prod(X[:, deg_pair], axis=1)

    return X_out

In [3]:
def scale_data(X):
    """ This function scales input data by feature separately """
    
    norms = np.linalg.norm(X, ord = 2, axis = -1)
    return X / np.expand_dims(norms, -1)

# Penalty terms for regression models

In [4]:
class lasso_penalty_term():
    """ Class of Lasso penalty term i.e 'absolute value of magnitude' """
    
    def __init__(self, lamb):
        self.lamb = lamb
    
    def __call__(self, w):
        return self.lamb * np.linalg.norm(w)

    def grad(self, w):
        return self.lamb * np.sign(w)

In [5]:
class poly_ridge_regression_penalty_term():
    """ Class of Polynomial Ridge Regression penalty term i.e 'squared magnitude' """
    
    def __init__(self, lamb):
        self.lamb = lamb
    
    def __call__(self, w):
        return (self.lamb / 2) * w.T @ w

    def grad(self, w):
        return self.lamb * w

In [6]:
class elastic_net_penalty_term():
    """ Class of Elastic Net penalty term """
    
    def __init__(self, lamb, l1_factor = 0.5):
        self.lamb = lamb
        self.l1_factor = l1_factor

    def __call__(self, w):
        l1_term = self.l1_factor * np.linalg.norm(w)
        l2_term = (1 - self.l1_factor)/2 * w.T @ w 
        return self.lamb * (l1_term + l2_term)

    def grad(self, w):
        l1_term = self.l1_factor * np.sign(w)
        l2_term = (1 - self.l1_factor) * w
        return self.lamb * (l1_term + l2_term)

# Main Regression Class

In [7]:
class Regression(object):
    """ Implementation of Regression Class that is super for LassoRegression, 
    PolynomialRegression and ElasticNetRegression. It uses one of the penalty term in fit function """
    
    def __init__(self, learning_rate, n_iterations):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def init_weights(self, n_features):
        self.w = np.random.randn(n_features)

    def fit(self, X, y):
        # Add bias parameters and initialize weights
        X = np.insert(X, 0, 1, axis = 1)
        
        self.init_weights(n_features = X.shape[1])
        
        self.loss = []
        
        # Gradient descend for n_iterations
        for i in tqdm(range(self.n_iterations)):
            y_pred = X @ self.w

            mse = np.mean(0.5 * (y - y_pred)**2 + self.regularization(self.w))
            self.loss.append(mse)
            
            grad_w = - (y - y_pred) @ X + self.regularization.grad(self.w)
            
            # Update the weights
            self.w -= self.learning_rate * grad_w
            
    def predict(self, X):
        # Add bias parameters
        X = np.insert(X, 0, 1, axis=1)
        y_pred = X @ self.w
        return y_pred

# Lasso Regression Class

In [8]:
class LassoRegression(Regression):
    """ Implementation of Lasso Regression Class that uses lasso penalty term 
                    i.e l1 penalty as regularization in fit function """
    
    def __init__(self, learning_rate = 1e-5, n_iterations = 100000, degree = 1, reg_factor = 1e-5):
        self.degree = degree
        self.regularization = lasso_penalty_term(lamb = reg_factor)
        super(LassoRegression, self).__init__(learning_rate, n_iterations)

    def fit(self, X, y):
        X = scale_data(add_poly_features(X, degree = self.degree))
        super(LassoRegression, self).fit(X, y)

    def predict(self, X):
        X = scale_data(add_poly_features(X, degree = self.degree))
        return super(LassoRegression, self).predict(X)

# Polynomial Ridge Regression Class

In [9]:
class PolynomialRidgeRegression(Regression):
    """ Implementation of Polynomial Ridge Regression that uses ridge regression penalty term
                    i.e l1 penalty as regularization in fit function """

    def __init__(self, learning_rate = 1e-5, n_iterations = 100000, degree = 1, reg_factor = 1e-5):
        self.degree = degree
        self.regularization = poly_ridge_regression_penalty_term(lamb = reg_factor)
        super(PolynomialRidgeRegression, self).__init__(learning_rate, n_iterations)

    def fit(self, X, y):
        X = scale_data(add_poly_features(X, degree = self.degree))
        super(PolynomialRidgeRegression, self).fit(X, y)

    def predict(self, X):
        X = scale_data(add_poly_features(X, degree = self.degree))
        return super(PolynomialRidgeRegression, self).predict(X)

# Elastic Net Regression Class

In [10]:
class ElasticNet(Regression):
    """ Implementation of Elastic Net Regression that uses elastic net penalty term in fit function """
    
    def __init__(self, learning_rate = 1e-5, n_iterations = 100000, degree = 1, reg_factor = 1e-5, l1_factor = 0.5):
        self.degree = degree
        self.regularization = elastic_net_penalty_term(lamb = reg_factor, l1_factor = l1_factor)
        super(ElasticNet, self).__init__(learning_rate, n_iterations)

    def fit(self, X, y):
        X = scale_data(add_poly_features(X, degree = self.degree))
        super(ElasticNet, self).fit(X, y)

    def predict(self, X):
        X = scale_data(add_poly_features(X, degree = self.degree))
        return super(ElasticNet, self).predict(X)

# Reading Prepared Data from first notebook

In [11]:
data = pd.read_csv('data_prepared.csv')
data = data.drop('Unnamed: 0', axis = 1)

In [12]:
X = data.drop('Price', axis = 1).values
y = data.Price.values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.1)

In [14]:
def RMSE(y1, y2):
    return np.sqrt(mean_squared_error(y1, y2))

# Initializing and Evaluating Regression Models

In [15]:
lasso = LassoRegression(n_iterations=1000000)
PolyRR = PolynomialRidgeRegression(n_iterations=1000000)
Elnet = ElasticNet(n_iterations=1000000)

## Lasso

In [16]:
lasso.fit(X_train, y_train)

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [01:11<00:00, 14026.37it/s]


In [17]:
y_pred = lasso.predict(X_train)
print("Lasso RMSE score on Train data: ", RMSE(y_pred, y_train))
y_pred = lasso.predict(X_test)
print("Lasso RMSE score on Test data: ", RMSE(y_pred, y_test))

Lasso RMSE score on Train data:  0.43127566895232505
Lasso RMSE score on Test data:  0.4208689070969225


## Elastic Net

In [18]:
Elnet.fit(X_train, y_train)

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [01:19<00:00, 12524.20it/s]


In [19]:
y_pred = Elnet.predict(X_train)
print("Elastic Net RMSE score on Train data: ", RMSE(y_pred, y_train))
y_pred = Elnet.predict(X_test)
print("Elastic Net RMSE score on Test data: ", RMSE(y_pred, y_test))

Elastic Net RMSE score on Train data:  0.4316505865248443
Elastic Net RMSE score on Test data:  0.42112741049947955


## Polynomial Ridge Regression

In [20]:
PolyRR.fit(X_train, y_train)

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [01:12<00:00, 13880.73it/s]


In [21]:
y_pred = PolyRR.predict(X_train)
print("Polynomial Ridge Regression RMSE score on Train data: ", RMSE(y_pred, y_train))
y_pred = PolyRR.predict(X_test)
print("Polynomial Ridge Regression RMSE score on Test data: ", RMSE(y_pred, y_test))

Polynomial Ridge Regression RMSE score on Train data:  0.43140859174186263
Polynomial Ridge Regression RMSE score on Test data:  0.4207223498683264


# Average Regression Models

In [22]:
class AverageModels(object):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        
        for model in self.models:
            model.fit(X, y)

        return self
    
    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=1)

In [23]:
average_models = AverageModels([lasso, Elnet, PolyRR])

In [24]:
average_models.fit(X_train, y_train)

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [01:06<00:00, 15059.48it/s]


In [25]:
y_pred = average_models.predict(X_train)
print("Averaged base models RMSE score on Train data: ", RMSE(y_pred, y_train))
y_pred = average_models.predict(X_test)
print("Averaged base models RMSE score on Test data: ", RMSE(y_pred, y_test))

Averaged base models RMSE score on Train data:  0.4314411204065008
Averaged base models RMSE score on Test data:  0.4215146365340259


### As we can see implemented models show nearly the same results, well not a surprise due to the fact that they have the same super class:)